In [1]:
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Preprocessing

Goal: Pre-process 2 datasets and 

In [2]:
train_actress = pd.read_json("input/actress.json")
train_movie = pd.read_csv("input/movies.csv")

In [3]:
train_actress.head()

,birthday,bust,height,hip,hobby,id,imageUrl,japanName,name,prefectures,siteUrl,waist
0,1980-12-13T16:00:00.000Z,95,0,88,クラシックバレエ,3,https://pics.dmm.co.jp/mono/actjpgs/kitagawa_y...,北川ゆい,Kitagawa Yui,,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,60
1,1978-08-14T16:00:00.000Z,90,157,86,水泳、ゴルフ,4,https://pics.dmm.co.jp/mono/actjpgs/akira.jpg,Akira,Akira,東京都,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,58
2,1982-07-06T16:00:00.000Z,96,168,85,パソコン,6,https://pics.dmm.co.jp/mono/actjpgs/cocolo.jpg,COCOLO,Cocolo,茨城県,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,55
3,1983-04-11T16:00:00.000Z,80,162,82,料理、大食い,8,https://pics.dmm.co.jp/mono/actjpgs/saiko.jpg,Saiko,Saiko,東京都,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,59
4,1970-06-17T16:00:00.000Z,82,162,89,エアロビクス,11,https://pics.dmm.co.jp/mono/actjpgs/aida_momo.jpg,あいだもも,Aida Momo,東京都,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,55


In [4]:
train_movie.head()

,actress,genre,maker,imageUrl,name,price,length,actress_id
0,"405,奥菜千春","顔射,単体作品,ローター,学生服,OL,レズ,コスプレ,ベスト・総集編,女子校生,バイブ,中出し","その他,other,（TQT）",{'list': 'http://pics.dmm.co.jp/digital/video/...,特選 奥菜千春,300,120,405
1,"1251,手塚美紗","単体作品,P・P,コスプレ,ベスト・総集編,バイブ,中出し","その他,other,（TQT）",{'list': 'http://pics.dmm.co.jp/digital/video/...,特選 手塚美紗,300,120,1251
2,"1956,川浜理奈","潮吹き,単体作品,P・P,パイパン,ぶっかけ","その他,other,ankylosaur",{'list': 'http://pics.dmm.co.jp/digital/video/...,潮吹剃毛 to shave！ 川浜理奈,300,45,1956
3,"2021,早川あゆみ","単体作品,縛り・緊縛,ごっくん,P・P,パイパン,バイブ","その他,other,ankylosaur",{'list': 'http://pics.dmm.co.jp/digital/video/...,強制剃毛 to shave！ 早川あゆみ,300,45,2021
4,"2472,藤丸らん","単体作品,ごっくん,P・P,パイパン,バイブ","その他,other,ankylosaur",{'list': 'http://pics.dmm.co.jp/digital/video/...,剃毛体験 to shave！ 藤丸らん,300,45,2472


Counting number of movies per actress by using movie data

In [ ]:
def _counting_movies(actress_id):
    return len(train_movie[train_movie.actress_id == actress_id])
train_actress['no_movies'] = train_actress['id'].apply(lambda x: _counting_movies(x))

Remove NAN values

In [ ]:
train_actress = train_actress[train_actress.no_movies != 0]
train_actress = train_actress[train_actress['birthday'].notnull()]

Calculate the Age

In [ ]:
train_actress['birthday'] = pd.to_datetime(train_actress['birthday'])
train_actress['age'] = train_actress['birthday'].apply(lambda x: pd.to_datetime('today').year - x.year)

### Process unicode columns

In [8]:
def extract_element_to_columns(column):
    tmp_list = list(set(','.join(column).split(',')))
    final_list = []
    for i, line in enumerate(tmp_list):
        try:
           u = line.strip().decode('utf-8')
        except UnicodeDecodeError as e:
            print 'decode failed at', e.args[2], 'leaving', repr(line[e.args[2]:])
        print u
        final_list.append(u)
    return final_list

Process the Genre

In [11]:
# genre_list = extract_element_to_columns(train_movie['genre'])
tmp_list = list(set(','.join(train_movie['genre']).split(',')))

In [15]:
for genre in tmp_list:
    train_movie[genre] = 

[u'\u604b\u611b',
 u'\u30e1\u30a4\u30c9',
 u'\u98df\u7cde',
 u'\u30ad\u30e3\u30f3\u30ae\u30e3\u30eb',
 u'\u4e3b\u89b3',
 u'\u30cf\u30a4\u30d3\u30b8\u30e7\u30f3',
 u'\u7af6\u6cf3\u30fb\u30b9\u30af\u30fc\u30eb\u6c34\u7740',
 u'AV OPEN  \u4e59\u5973\u90e8\u9580',
 u'\u52d5\u753b',
 u'\u3081\u304c\u306d',
 u'\u9854\u5c04',
 u'\u30d1\u30e9\u30c0\u30a4\u30b9TV',
 u'\u30b3\u30e9\u30dc\u4f5c\u54c1',
 u'\u3075\u305f\u306a\u308a',
 u'AV OPEN  \u5973\u512a\u90e8\u9580',
 u'\u62f7\u554f',
 u'\u6642\u9593\u4ee5\u4e0a\u4f5c\u54c1',
 u'\u7279\u64ae',
 u'\u5fa9\u523b',
 u'AV OPEN  \u30b9\u30fc\u30d1\u30fc\u30d8\u30d3\u30fc\u7d1a',
 u'\u89e6\u624b',
 u'\u65e5\u713c\u3051',
 u'\u4f53\u9a13\u544a\u767d',
 u'\u7570\u7269\u633f\u5165',
 u'\u5973\u5b50\u5927\u751f',
 u'\u8d85\u4e73',
 u'\u7fa9\u6bcd',
 u'SF',
 u'\u8131\u7cde',
 u'\u51e6\u5973',
 u'\u30e2\u30c7\u30eb',
 u'\u4f53\u64cd\u7740\u30fb\u30d6\u30eb\u30de',
 u'\u5c40\u90e8\u30a2\u30c3\u30d7',
 u'\u30a2\u30af\u30b7\u30e7\u30f3',
 u'\u5973\u533b',
 u'

Process the maker

In [ ]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
train_movie['maker'] = train_movie['maker'].str.encode('utf-8')

In [ ]:
# maker = train_movie['maker'].apply(lambda x: ''.join(x))
maker = ','.join(train_movie.maker)

In [ ]:
maker = maker.split(",")

In [ ]:
maker[2]

In [ ]:
genre = ','.join(train_movie['genre'])

In [ ]:
list_genre = genre.split(',')

In [ ]:
list_genre = list(set(list_genre))

In [ ]:
len(list_genre)

In [ ]:
for i, line in enumerate(list_genre):
    print '---- line', i
    try:
        u = line.strip().decode('utf-8')
    except UnicodeDecodeError as e:
        print 'decode failed at', e.args[2], 'leaving', repr(line[e.args[2]:])
        u = line.strip().decode('utf-8', 'ignore')
    print u